In [ ]:
import random
def capitalize(text):
    new_text = ''
    vals = [True, False]
    weights = (0.01, 0.99)
    for i in range(len(text)):
        if random.choices(vals, weights=weights, k=1)[0]:
            new_text += text[i].upper()
        else:
            new_text += text[i]
    return new_text

In [ ]:
keys = "1234567890-= qwertyuiop[]\ asdfghjkl;' zxcvbnm,./"
keys = keys.split(' ')
keyboard_layout = [list(keyset) for keyset in keys]
maxLen = 0
for line in keyboard_layout:
    if maxLen < len(line):
        maxLen = len(line)
for i in range(len(keyboard_layout)):
    for _ in range(maxLen - len(keyboard_layout[i])):
        keyboard_layout[i].append(' ')

keyboard_layout.insert(0, [' ' for _ in range(13)])
keyboard_layout.append([' ' for _ in range(13)])
for i in range(len(keyboard_layout)):
    keyboard_layout[i].append(' ')
    keyboard_layout[i].insert(0, ' ')
# for i in keyboard_layout:
#     print(i)
neighbors = {}

for i in range(1, len(keyboard_layout)-1):
    for j in range(1, len(keyboard_layout[i])-1):
        temp = [
                keyboard_layout[i][j+1],
                keyboard_layout[i][j-1],
                keyboard_layout[i-1][j],
                keyboard_layout[i+1][j],
                keyboard_layout[i+1][j+1],
                keyboard_layout[i-1][j+1],
                keyboard_layout[i+1][j-1],
                keyboard_layout[i-1][j-1],
                ]
        neighbors[keyboard_layout[i][j]] = [t for t in temp if t != ' ']
# print(neighbors)


def butterfinger(text):
    new_text = ''
    vals = [True, False]
    weights = (0.03, 0.97)
    for i in range(len(text)):
        if random.choices(vals, weights=weights, k=1)[0]:
            try:
                new_text += random.choice(neighbors[text[i]])
            except:
                new_text += text[i]
        else:
            new_text += text[i]
    return new_text

In [ ]:
!pip install inflect
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import inflect
from nltk.tokenize import word_tokenize
p = inflect.engine()

def num_to_words(text):
    new_text = ''
    vals = [True, False]
    weights = [0.99, 0.01]
    # tokenized_text = [str(i) for i in nlp(text).sents]
    tokenized_text = [str(i) for i in word_tokenize(text)]
    #print(tokenized_text)
    for word in tokenized_text:
        if word.isnumeric():
            if random.choices(vals, weights=weights, k=1)[0]:
                new_text += p.number_to_words(word) + ' '
            else:
                new_text += word + ' '
        else:
            new_text += word + ' '
    return new_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# create input data and pass it through functions above
sentence = 'There are 50 states in USA.'
# iterate data over the following functions
print(num_to_words(sentence))
print(butterfinger(sentence))
print(capitalize(sentence))

There are fifty states in USA . 
There are 50 states in USA.
ThEre are 50 states in USA.


In [ ]:
# Before you run this, make sure that the shared CSE 573: Stance Detection folder is on your My Drive tab. You can do this by right clicking on the folder and selecting "Add shortcut to Drive"
from google.colab import drive
from google.colab import files
from tqdm.notebook import tqdm
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Progress Bar
tqdm().pandas()

# Open Procon20
proconfile_train = open('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/Procon20/procon_final.csv', encoding='utf8', errors='backslashreplace')
procon_train = pd.read_csv(proconfile_train,header=0, sep=',')

# Create a copy and backtranslate the tweets
procon_train_copy_num = procon_train.copy()
procon_train_copy_butter = procon_train.copy()
procon_train_copy_capital = procon_train.copy()
for i, row in tqdm(procon_train_copy_num.iterrows(), total=procon_train_copy_num.shape[0]):
  procon_train_copy_num.at[i, 'Query'] = num_to_words(row['Query'])
  procon_train_copy_butter.at[i, 'Query'] = butterfinger(row['Query'])
  procon_train_copy_capital.at[i, 'Query'] = capitalize(row['Query'])

# Open SemEval-2016
txtfile = open('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/SemEval-2016/data-all-annotations/trainingdata-all-annotations.txt', encoding='utf8', errors='backslashreplace')
semeval_train = pd.read_csv(txtfile, sep='\t')

# Change Stance to 0, 1, or 2
for i, row in semeval_train.iterrows():
  if row['Stance'] == 'AGAINST':
    semeval_train.at[i, 'Stance'] = 0
  elif row['Stance'] == 'FAVOR':
    semeval_train.at[i, 'Stance'] = 1
  else:
    semeval_train.at[i, 'Stance'] = 2

# Drop Unnecessary Attributes
semeval_train = semeval_train.drop(['ID', 'Opinion towards', 'Sentiment'], axis=1)

# Rename Tweet to Query
semeval_train.rename(columns={'Tweet': 'Query'}, inplace=True)

# Create a copy and backtranslate the tweets
semeval_train_copy_num = semeval_train.copy()
semeval_train_copy_butter = semeval_train.copy()
semeval_train_copy_capital = semeval_train.copy()
for i, row in tqdm(semeval_train_copy_num.iterrows(), total=semeval_train_copy_num.shape[0]):
  semeval_train_copy_num.at[i, 'Query'] = num_to_words(row['Query'])
  semeval_train_copy_butter.at[i, 'Query'] = butterfinger(row['Query'])
  semeval_train_copy_capital.at[i, 'Query'] = capitalize(row['Query'])

Mounted at /content/drive


0it [00:00, ?it/s]

  0%|          | 0/5984 [00:00<?, ?it/s]

  0%|          | 0/2814 [00:00<?, ?it/s]

In [ ]:
# Combine all of these
combined = pd.concat([procon_train_copy_num, procon_train_copy_butter, procon_train_copy_capital, semeval_train_copy_num, semeval_train_copy_butter, semeval_train_copy_capital])

# Save the combined dataframe to Drive
combined.to_csv('/content/drive/My Drive/CSE 573: Stance Detection /Datasets/Both/minor_changes.csv', index=False)